In [3]:
import torchvision.models as models
model = models.vgg16(pretrained=True, progress=True)

In [6]:
from __future__ import division

import random
import os
from os import listdir
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
import torchvision
from torchvision import models
import torchvision.datasets as dset
import torchvision.transforms as T
from torch.utils.data import TensorDataset,Dataset,DataLoader
from tqdm.auto import tqdm

from torch.utils.data.sampler import SequentialSampler
from torchvision import transforms

import argparse
import pydicom
import glob
from matplotlib import pyplot as plt
from skimage.color import gray2rgb
import functools
import pydicom
import seaborn as sns
import scipy
import PIL

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [15]:
data_dir = '/projectnb/ece601/kaggle-pulmonary-embolism/rsna-str-pulmonary-embolism-detection/'
train_csv = data_dir + 'train.csv'
train_dir = data_dir + 'train/'

train = pd.read_csv(train_csv)
#test_data  = pd.read_csv(data_dir + "/test.csv")
#sample = pd.read_csv(data_dir + "/sample_submission.csv")

cols_ID = ["StudyInstanceUID","SeriesInstanceUID","SOPInstanceUID"]
train["ImagePath"] =train_dir+ train[cols_ID[0]]+"/"+train[cols_ID[1]]+"/"+train[cols_ID[2]]+".dcm"

In [9]:
print("{}:{}".format("Number of rows in train data",train.shape[0]))
print("{}:{}".format("Number of columns in train data",train.shape[1]))

Number of rows in train data:1790594
Number of columns in train data:17


In [10]:
train.head()

,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,leftsided_pe,chronic_pe,true_filling_defect_not_pe,rightsided_pe,acute_and_chronic_pe,central_pe,indeterminate
0,6897fa9de148,2bfbb7fd2e8b,c0f3cb036d06,0,0,0,0,0,0,1,1,0,0,1,0,0,0
1,6897fa9de148,2bfbb7fd2e8b,f57ffd3883b6,0,0,0,0,0,0,1,1,0,0,1,0,0,0
2,6897fa9de148,2bfbb7fd2e8b,41220fda34a3,0,0,0,0,0,0,1,1,0,0,1,0,0,0
3,6897fa9de148,2bfbb7fd2e8b,13b685b4b14f,0,0,0,0,0,0,1,1,0,0,1,0,0,0
4,6897fa9de148,2bfbb7fd2e8b,be0b7524ffb4,0,0,0,0,0,0,1,1,0,0,1,0,0,0


In [ ]:
def dist(column,color):
    sns.distplot(meta_data[column],label=column,color=color)
    plt.legend()

In [ ]:
def show_image(train=True):
    image_path = train["ImagePath"].sample(n=1).values[0]
    print(f"{y_} Image {r_}{image_path}")
    image = dicom.dcmread(image_path)
    image = image.pixel_array
    plt.figure(figsize=(7,7))
    plt.imshow(image,cmap='gray')
    plt.axis('off')
    plt.show()
show_image()

In [22]:
!pip install vtk
from sklearn.model_selection import KFold

import vtk
from vtk.util import numpy_support
from tqdm.auto import tqdm

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 103.4 MB 60.9 MB/s eta 0:00:01  |█▎                              | 4.2 MB 5.1 MB/s eta 0:00:20     |█▉                              | 5.9 MB 5.1 MB/s eta 0:00:20     |███████████████▍                | 49.6 MB 5.1 MB/s eta 0:00:11     |████████████████▊               | 54.1 MB 5.1 MB/s eta 0:00:10     |███████████████████████▋        | 76.2 MB 60.9 MB/s eta 0:00:01
You should consider upgrading via the '/share/pkg.7/python3/3.7.7/install/bin/python3.7 -m pip install --upgrade pip' command.


In [10]:
!pip install albumentations
import cv2
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/pkg.7/python3/3.7.7/install/bin/python3.7 -m pip install --upgrade pip' command.
['/projectnb2/ece601/kaggle-pulmonary-embolism/jiamingy', '/share/pkg.7/vtk/9.0.1/install/lib/SCC', '', '/share/pkg.7/pytorch/1.5.1/install/3.7/lib/python3.7/site-packages', '/share/pkg.7/opencv/4.0.1/install/lib/site-packages', '/share/pkg.7/python3/3.7.7/install/lib/python37.zip', '/share/pkg.7/python3/3.7.7/install/lib/python3.7', '/share/pkg.7/python3/3.7.7/install/lib/python3.7/lib-dynload', '/usr3/graduate/jiamingy/.local/lib/python3.7/site-packages', '/share/pkg.7/python3/3.7.7/install/lib/python3.7/site-packages', '/share/pkg.7/python3/3.7.7/install/lib/python3.7/site-packages/numpy-1.18.4-py3.7-linux-x86_64.egg', '/share/pkg.7/python3/3.7.7/install/lib/python3.7/site-packages/scipy-1.4.1-py3.7-linux-x86_64.egg', '/share/pkg.7/python3/3.7.7/install/lib/python3.7/site-packages/

ImportError: ERROR: recursion is detected during loading of "cv2" binary extensions. Check OpenCV installation.

In [23]:
class TrainData(Dataset):

    def __init__(self, dataframe, image_dir, transforms):
        super().__init__()
        
        self.df = dataframe
        self.image_ids = dataframe['image_path'].unique()
        self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, idx: int):
        #Generates one sample of the data
        image_id = self.image_ids[idx]
        image = pydicom.dcmread(f'{self.image_dir}{image_id}.dcm') #Read .dcm file into the loop
        image = image.reshape((512,512,1)).astype('float')

        labels =self.df[['negative_exam_for_pe', 'rv_lv_ratio_gte_1', 'rv_lv_ratio_lt_1',
                     'leftsided_pe', 'chronic_pe', 'rightsided_pe',
                     'acute_and_chronic_pe', 'central_pe', 'indeterminate']].loc[idx].values
        
        if self.transforms:
            image = {"image" : image,} #Create a dictionary of the image values. We can only force the kwargs through transform if they are in dict type
            image = self.transforms(**image)
            
        return image, labels #Return the transformed image, and the associated labels.
    
    def __len__(self) -> int:
        #The loader needs to know how many items we have in our dataset. Therefore we use the __len__ function as an upper-bound.
        return len(self.image_ids)

In [19]:
SEED  = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASSEED']  = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [24]:
target_columns = ['pe_present_on_image', 'negative_exam_for_pe', 'rv_lv_ratio_gte_1', 
                  'rv_lv_ratio_lt_1','leftsided_pe', 'chronic_pe','rightsided_pe', 
                  'acute_and_chronic_pe', 'central_pe', 'indeterminate']

#vtk is used because dicom is giving some error

reader = vtk.vtkDICOMImageReader()
def get_img(path):
    reader.SetFileName(path)
    reader.Update()
    _extent = reader.GetDataExtent()
    ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, _extent[5]-_extent[4]+1]

    ConstPixelSpacing = reader.GetPixelSpacing()
    imageData = reader.GetOutput()
    pointData = imageData.GetPointData()
    arrayData = pointData.GetArray(0)
    ArrayDicom = numpy_support.vtk_to_numpy(arrayData)
    ArrayDicom = ArrayDicom.reshape(ConstPixelDims, order='F')
    ArrayDicom = cv2.resize(ArrayDicom,(512,512))
    return ArrayDicom


def convert_to_rgb(array):
    array = array.reshape((512, 512, 1))
    return np.stack([array, array, array], axis=2).reshape((3,512, 512))

In [25]:
class RsnaDataset(Dataset):
    
    def __init__(self,df,transforms=None):
        super().__init__()
        self.image_paths = df['ImagePath'].unique()
        self.df = df
        self.transforms = transforms
    
    def __getitem__(self,index):
        
        image_path = self.image_paths[index]
        data = self.df[self.df['ImagePath']==image_path]
        labels = data[target_columns].values.reshape(-1)
        image = get_img(image_path)
        image = convert_to_rgb(image)
        
        if self.transforms:
            image = self.transforms(image=image)['image']
            
        image = torch.tensor(image,dtype=torch.float)        
        labels = torch.tensor(labels,dtype=torch.float)
        
        return image,labels
           
    def __len__(self):
        return self.image_paths.shape[0]  

In [26]:
classes = len(target_columns)
#in_features = model.fc.in_features
#model.fc = nn.Linear(in_features,classes)
model.classifier._modules['6'] = nn.Linear(4096, classes)

In [33]:
config={
       "learning_rate":0.001,
       "train_batch_size":32,
        "valid_batch_size":32,
        "test_batch_size":64,
       "epochs":10,
       "nfolds":3,
       "number_of_samples":7000
       }

In [50]:
#https://pytorch.org/hub/pytorch_vision_vgg/
transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [38]:
def test():
    model.eval()
    all_prediction = np.zeros((test_data.shape[0],len(target_columns)))
    study_ids = list()
    sop_ids = list()
    for i in range(config["nfolds"]):
        #model.load_state_dict(torch.load(f"{model_path}model{i}.bin"))
        predictions = list()
        model.to(device)
        test_ds = RsnaDataset(test_data)
        test_dl = DataLoader(test_ds,
                        batch_size=config['test_batch_size'],
                        shuffle=False)
        
        tqdm_loader = tqdm(test_dl)
        
        with torch.no_grad():
            for inputs in tqdm_loader:
                images = inputs["image"].to(device, dtype=torch.float)
                outputs = model(images) 
                predictions.extend(outputs.cpu().detach().numpy())
                if i == 0:
                    study_ids.extend(inputs["study_id"])
                    sop_ids.extend(inputs["sop_id"])

        all_prediction += np.array(predictions)/config['nfolds']
        
    return all_prediction

In [45]:
sample = train.sample(n=config["number_of_samples"]).reset_index(drop=True)
test_data = sample

In [46]:
predictions = test()

NameError: name 'cv2' is not defined